In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"
import keras
import numpy as np
import io
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow import keras

epochs = 120


# Learning rate scheduler
def cosine_annealing_scheduler(epoch, lr):
    initial_lr = 1e-3
    min_lr = 1e-6
    T_max = int(epochs / 2)

    cosine_decay = 0.5 * (1 + np.cos(np.pi * (epoch % T_max) / T_max))
    new_lr = (initial_lr - min_lr) * cosine_decay + min_lr

    return float(new_lr)


import numpy as np
from keras.utils import image_dataset_from_directory


# Load datasets using Keras utilities
batch_size = 32
img_size = (512, 512)

# train_ds = image_dataset_from_directory(
#     "official_data/train",
#     label_mode="categorical",
#     image_size=img_size,
#     batch_size=batch_size,
#     color_mode="grayscale",
# )

val_ds = image_dataset_from_directory(
    "official_data/valid",
    label_mode="categorical",
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale",
)

test_ds = image_dataset_from_directory(
    "official_data/test",
    label_mode="categorical",
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale",
)

In [ ]:
def dataset_stats(dataset):
    pixel_values = []
    for images, _ in dataset:
        pixel_values.extend(images.numpy().flatten())
    
    return {
        'mean': np.mean(pixel_values),
        'std': np.std(pixel_values),
        'min': np.min(pixel_values),
        'max': np.max(pixel_values),
        'histogram': np.histogram(pixel_values, bins=256, range=(0, 255))[0]
    }

# train_stats = dataset_stats(train_ds)
val_stats = dataset_stats(val_ds)
test_stats = dataset_stats(test_ds)

# print("Train stats:", train_stats)
print("Val stats:", val_stats)
print("Test stats:", test_stats)

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(val_stats['histogram'], alpha=0.5, label='Val', bins=256, range=(0, 255))
plt.hist(test_stats['histogram'], alpha=0.5, label='Test', bins=256, range=(0, 255))
plt.legend()
plt.title("Pixel Value Distribution")
plt.show()

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def extract_features(dataset, sample_size=1000):
    features = []
    labels = []
    for batch_images, batch_labels in dataset.take(sample_size // batch_size + 1):
        flattened = batch_images.numpy().reshape(len(batch_images), -1)  # Flatten images
        features.extend(flattened)
        labels.extend(np.argmax(batch_labels.numpy(), axis=1))
    return np.array(features), np.array(labels)

# Sample subsets (for speed)
X_val, y_val = extract_features(val_ds)
X_test, y_test = extract_features(test_ds)

# PCA for visualization
pca = PCA(n_components=2)
X_val_pca = pca.fit_transform(X_val)
X_test_pca = pca.transform(X_test)

plt.figure(figsize=(10, 6))
plt.scatter(X_val_pca[:, 0], X_val_pca[:, 1], alpha=0.3, marker='x', label='Val', c=y_val)
plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], alpha=0.3, marker='^', label='Test', c=y_test)
plt.legend()
plt.title("PCA of Train/Val/Test Sets")
plt.show()